In [428]:
import day8
import numpy as np

In [429]:
def encode(st):
    return np.unpackbits(np.array([2**(ord(letter)-97) for letter in st]).sum().astype('uint8'))[1:]


def load_data(path):
    with open(path) as file:
        tokens = file.read().replace(' | ', ' ').replace('\n', ' ').split(' ')
    tokens = list(map(encode, tokens))
    array = np.array(tokens)
    return array.reshape(int(array.size/14/7), 14, 7)


In [445]:
data = load_data('../data/day8/input.txt')
no_readings = data.shape[0]
digits = data[:,:10]

In [447]:
ones = digits[digits.sum(axis=2) ==2]
sevens = digits[digits.sum(axis=2) ==3]
fours = digits[digits.sum(axis=2) ==4]
eights = digits[digits.sum(axis=2) ==7]

In [448]:
can_be_2_5_3 = digits[digits.sum(axis=2) ==5].reshape(no_readings,3,7)
mask_3 = (can_be_2_5_3 * ones[:,np.newaxis,:]).sum(axis=2) ==2
threes = can_be_2_5_3[mask_3]

In [449]:
can_be_0_6_9 = digits[digits.sum(axis=2) ==6].reshape(no_readings,3,7)
mask_9 = (can_be_0_6_9 & threes[:,np.newaxis,:] & fours[:,np.newaxis,:]).sum(axis=2) == 3
nines = can_be_0_6_9[mask_9]

In [450]:
can_be_0_6 = can_be_0_6_9[~mask_9].reshape(no_readings,2,7)
mask_zeros = (can_be_0_6 & ones[:,np.newaxis,:]).sum(axis=2) == 2
zeroes = can_be_0_6[mask_zeros]
sixes = can_be_0_6[~mask_zeros]

In [451]:
mask_5 = (can_be_2_5_3 & sixes[:,np.newaxis,:]).sum(axis=2) ==5
fives = can_be_2_5_3[mask_5]

In [452]:
mask_2 = ~mask_5 & ~mask_3
twos = can_be_2_5_3[mask_2]

In [453]:
ordered_digits = np.concatenate([zeroes, ones,twos, threes, fours, fives, sixes, sevens, eights, nines], axis=1) \
    .reshape(no_readings,10,7)

packed_digits = np.packbits(ordered_digits, axis=2)

In [454]:
readings = data[:,10:]
packed_readings = np.packbits(readings,axis=2)

In [455]:
correct_readings = ((packed_digits).reshape(no_readings,10,1) - packed_readings.reshape(no_readings,1,4))

In [456]:
correct_readings[correct_readings != 0] = -1

In [457]:
t = np.where((correct_readings + 1))
x = t[1].reshape(no_readings,4)
y = t[2].reshape(no_readings,4)

end_result = x[:,np.argsort(y)].diagonal().T
end_result = (end_result * (10** np.arange(3,-1,-1))).sum(axis=1)
end_result.sum()

989396